# 获取东方财富增发列表

In [105]:
import requests
import re
import json
import demjson
import pandas as pd
import time
import tqdm
import datetime

In [ ]:
page=1

In [39]:
def get_page(page):
    url= 'http://datainterface.eastmoney.com/EM_DataCenter/JS.aspx?type=SR&sty=ZF&st=5&sr=-1&p='+str(page)+'&ps=50&js=var%20GoBTBAVo={pages:(pc),data:[(x)]}&stat=1&rt=53682125'
    r = requests.get(url)
    data = demjson.decode(re.findall("{.*}",r.text)[0])
    res = [i.split(',')[:8] for i in data['data']]
    df = pd.DataFrame(res)
    df.columns = ['code','name','type','stock_num','release_price','current_price','release_date','open_date']
    return df

In [43]:
df_ls = []

In [44]:
for page in tqdm.tqdm(range(1,84)):
    tmp_df = get_page(page)
    df_ls.append(tmp_df)
    time.sleep(1)

100%|██████████| 83/83 [02:11<00:00,  1.58s/it]


In [94]:
df = pd.concat(df_ls)

df.reset_index(inplace=True)

df['release_price'] = df['release_price'].astype('float')

df['current_price'] = [float(i) if i else 0 for i in df['current_price'] ]

df.drop(columns=['index'],inplace=True)

df.drop_duplicates(inplace=True)

In [100]:
df['release_date'] = pd.to_datetime(df['release_date'])

In [101]:
df['open_date'] = pd.to_datetime(df['open_date'])

# 过滤满足条件的股票

In [108]:
# 过滤最近半年的增发
flag_date = df['release_date']>'2020-06-01'

In [109]:
# 当前股价大于30
flag_current_price = df['current_price']>30

In [110]:
# 当前股价小于增发价
flag_current_le_release = df['current_price']<df['release_price']*1.1

In [111]:
df_res = df[flag_date&flag_current_price&flag_current_le_release]

In [112]:
df_res.shape

(21, 8)

In [114]:
df_res['ratio'] = df['current_price']/df['release_price']

/Users/taptap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [115]:
df_res

,code,name,type,stock_num,release_price,current_price,release_date,open_date,ratio
33,300502,新易盛,定向增发,31226336,52.84,52.20,2020-12-28,2020-12-28,0.987888
38,688001,华兴源创,定向增发,9450355,35.66,36.39,2020-12-25,2020-12-25,1.020471
66,300785,值得买,定向增发,8196437,88.88,82.14,2020-12-10,2020-12-22,0.924167
72,300620,光库科技,定向增发,16888677,42.04,39.80,2020-12-08,2020-12-18,0.946717
76,603990,麦迪科技,定向增发,19863488,36.63,31.85,2020-12-04,2020-12-04,0.869506
88,603078,江化微,定向增发,8787878,33.00,31.77,2020-11-27,2020-11-27,0.962727
100,300738,奥飞数据,定向增发,12310848,38.99,33.20,2020-11-18,2020-11-19,0.851500
112,002940,昂利康,定向增发,5946819,46.26,36.08,2020-11-12,2020-11-27,0.779939
113,603658,安图生物,定向增发,20375760,151.16,152.79,2020-11-12,2020-11-12,1.010783
142,002683,宏大爆破,定向增发,43037080,41.07,42.20,2020-10-27,2020-10-30,1.027514


In [116]:
df_res.to_excel('增发列表.xlsx',index=False)